In [1]:
import pytz
from datetime import datetime
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
from requests.adapters import HTTPAdapter, Retry

requests.adapters.DEFAULT_POOLSIZE = 100
retries = Retry(total=20)
# 세션을 생성하고 ConnectionPool 크기를 설정
session = requests.Session()
session.mount('https://', requests.adapters.HTTPAdapter(pool_connections=100, pool_maxsize=100, max_retries=retries))

import pytz
from datetime import datetime
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
from requests.adapters import HTTPAdapter, Retry


# soup = BeautifulSoup(html, 'html.parser')
# items = soup.find_all('option')

In [40]:
timestamp_fixed_value = 1709688760481
timestamp_add_count = 0
def get_json_result(iso_code, page):
    headers = {
        "accept": "application/json, text/javascript, */*; q=0.01",
        "accept-language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
        "content-type": "application/x-www-form-urlencoded; charset=UTF-8",
        "sec-ch-ua": "\"Not_A Brand\";v=\"8\", \"Chromium\";v=\"120\", \"Google Chrome\";v=\"120\"",
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-platform": "\"macOS\"",
        "sec-fetch-dest": "empty",
        "sec-fetch-mode": "cors",
        "sec-fetch-site": "same-origin",
        "x-requested-with": "XMLHttpRequest",
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    }

    global timestamp_add_count
    timestamp_add_count += 1
    ts = timestamp_fixed_value + timestamp_add_count
    response = requests.get(f"https://unipass.customs.go.kr/csp/myc/bsopspptinfo/rgsrentsmt/OvcsSgnQryCtr/retrieveOvcsSgn.do?pageIndex={page}&pageUnit=100&vOvcsConmCheck=&ovcsSgn={iso_code}%25&ovrsAthrNo=&ovcsConm=&ovcsNatCd=&ovcsNatNm=&custSgnUseStcd=&_={ts}"
                            , headers=headers)
    response.raise_for_status()
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"[{response.status_code}] {response.text}")

In [51]:
def processing(j_data, iso_code, page):
    searchVo = j_data["searchVo"]
    count = j_data["count"]

    df = pd.DataFrame(j_data["resultList"])
    df_converted = df.astype(str).replace('nan', pd.NA)

    df_converted["iso_country_code"] = iso_code

    # df_converted.to_gbq("grainscanner.external_ingestion.unipass_foreign_customer_raw",
    #           "grainscanner",
    #             if_exists="append",
    #             location="asia-northeast3"
    #           )

    df_converted.to_parquet(f"gs://greenlabs-data-grainscanner/crawling/temp/unipass/ft_{iso_code}_{page}.parquet")

    return searchVo["lastIndex"] < count # 다음 페이지 가져와야하는지 여부

In [34]:
from google.cloud import bigquery

bigquery_client = bigquery.Client(project="grainscanner")

country_df = bigquery_client.query("""
select iso_code from `grainscanner.dw_meta.dim_iso_country` order by iso_code
""").to_dataframe()


In [ ]:
import time
error_list = []

is_check_point = False
for index, row in country_df.iterrows():
    page = 1
    if not is_check_point:
        if row["iso_code"] == "CA":
            page = 179
            is_check_point = True
        else:
            continue
        
    while True:
        print(row["iso_code"], page)
        try :
            j_data = get_json_result(row["iso_code"], page)

            if not processing(j_data, row["iso_code"], page):
                break
            time.sleep(1)
        except Exception as e:
            print(f' ** error : {row["iso_code"]} - {page} / {str(e)}')
            error_list.append({
                "iso_code": row["iso_code"],
                "page": page,
                "error": str(e)
            })
            time.sleep(3)

        page += 1

# iso 국가 코드 추출

In [6]:
import pandas as pd


df1 = pd.read_excel('/Users/yoonhae/Downloads/SINGLE_20240306_160117.xlsx')
df2 = pd.read_excel('/Users/yoonhae/Downloads/SINGLE_20240306_160158.xlsx')
df3 = pd.read_excel('/Users/yoonhae/Downloads/SINGLE_20240306_160227.xlsx')


In [10]:
result_df = pd.concat([df1, df2, df3], axis=0)
df_renamed = result_df.drop(columns=['LABEL-1']).rename(columns={
    'LABEL-2': 'iso_code',
    'LABEL-3': 'name_kr',
    'LABEL-4': 'name_en',
    'LABEL-5': 'name_abbr'
})

,LABEL-1,LABEL-2,LABEL-3,LABEL-4,LABEL-5
0,1,AD,안도라,Andorra,ANDORA
1,2,AE,아랍에미리트 연합,United Arab Emirates,U.A.E
2,3,AF,아프카니스탄,Afghanistan,AFGHAN
3,4,AG,안티가 바부다,Antigua and Barbuda,ANTG-BA
4,5,AI,앙귈라,Anguilla,ANGUILA
...,...,...,...,...,...
49,250,Z1,국제통화기금,International Monetary Fund,IMF
50,251,ZA,남아프리카공화국,South Africa,S.AFR
51,252,ZM,잠비아,Zambia,ZAMBIA
52,253,ZW,짐바브웨,Zimbabwe,ZIMBABW


In [36]:
df_renamed[df_renamed['iso_code'].isnull()]
# iso_code 에 NA 값이 있어야하는데 누락됨. 아마 Nan 으로 받아들인듯... -_-
# 그래서 강제로 NA 값을 넣어줌.
df_renamed['iso_code'].fillna('NA', inplace=True)


,iso_code,name_kr,name_en,name_abbr


In [17]:

df_renamed.to_gbq("grainscanner.dw_meta.dim_iso_country",
                  "grainscanner",
                    if_exists="append",
                    location="asia-northeast3"
                    )

100%|██████████| 1/1 [00:00<00:00, 6853.44it/s]


시간 차이: -3시간 37분 27초
